# PICSUM- Deployment

## Downloading the prerequisites

In [1]:
!sudo apt install tesseract-ocr
!pip install pytesseract
!git clone https://github.com/dhaneshragu/text2questions.git
%cd text2questions/fill-ups
!pip install streamlit
!pip install transformers 
!pip install sentencepiece
!pip install datasets
!git clone https://github.com/Nidhi15-02/question_generation.git
!pip install gradio

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 34 not upgraded.
Need to get 4,850 kB of archives.
After this operation, 16.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1 [1,598 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr amd64 4.1.1-2build2 [262 kB]
Fetched 4,850 kB in 2s (2,967 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/Fro

In [2]:
import gradio as gr
from PIL import Image
import os
import pytesseract
import torch
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.cluster.util import cosine_distance
import networkx as nx
# from transformers import PegasusForConditionalGeneration, PegasusTokenizer
# from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import pipeline as transformerspl
from helper import *
import re


if torch.cuda.is_available():
   device = torch.device("cuda")
else:
   device = torch.device("cpu")


summarizer = transformerspl("summarization", model="facebook/bart-large-cnn")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2023-06-02 15:41:32.672 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/ipykernel_launcher.py [ARGUMENTS]


## Creating Functions

In [3]:
def read(filepath):
    return pytesseract.image_to_string(Image.open(filepath))
    
def clean_text(text):
  article = text.split(".")
  article=[sentence for sentence in article if sentence!=""]
  # print(article)

  sentences = []

  for sentence in article:
      #print(sentence)
      sentence=sentence.replace(",", " , ").replace("'", " ' ").split(" ")
      #sentence=sentence.replace("[^a-zA-Z]", " ").split(" ")
      sentence=[word for word in sentence if word!=""]
      sentences.append(sentence)
    
  return sentences

def sentence_similarity(sent1, sent2, stopwords):   #Creating words in sentences to one hot encoding and then finding cosine distance between the vectors inorder to measure closeness
  
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)


def build_similarity_matrix(sentences, stop_words):
  
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix

def sentences(text, top_n='2'):

    if top_n==  None or top_n=="":
      top_n=2
    top_n=int(top_n)
    # Step 1 - Clean text to generate sentences

    sentences=clean_text(text)
    stop_words = stopwords.words('english')
    stop_words.append(".")
    stop_words.append(",")
    summarize_text = []

    # Step 2 - Generate Similary Martix across sentences

    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)
    # print(sentence_similarity_martix)

    # Step 3 - Rank sentences in similarity martix

    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    # print(sentence_similarity_graph)

    scores = nx.pagerank(sentence_similarity_graph)
    # print(scores)

    # Step 4 - Sort the rank and pick top sentences

    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    #Sorting the scores in decending order
    # print("Indexes of top ranked_sentence order are ", ranked_sentence)   

    for i in range(top_n):
      ranked_sentence[i][1][0]=ranked_sentence[i][1][0].capitalize()    #Capitalising 1st letter of sentence
      # print(ranked_sentence[i][1][0]) 
      summarize_text.append(" ".join(ranked_sentence[i][1]))

    # Step 5 - Offcourse, output the summarized text

    extractive_summarized=". ".join(summarize_text).replace(" , ",", ").replace(" ' ","'") + "."
    return extractive_summarized

def important_sentences(extractedInformation, no_of_sentences):
  extractive_summary=sentences(extractedInformation, no_of_sentences)
  text=""
  for index,sent in enumerate(extractive_summary.split(".")):
    if sent!='':text+=str(index+1)+". "+str(sent).strip()+".\n\n"
  return text
    
def summarize(extractedInformation):
  output = summarizer(extractedInformation, max_length=300, min_length=30, do_sample=False)
  return output[0]["summary_text"]

def fill_in_the_gaps(text,no=5):
  question_df = get_fillups(text)[['answer','final_text']].reset_index(drop=True)
  fillup_df={'question':[],'answer':[]}
  for i in range(len(question_df)):
        if re.search(re.escape(question_df.answer.iloc[i]),question_df.final_text.iloc[i], flags=re.IGNORECASE):
            reg = re.compile(re.escape(question_df.answer.iloc[i]), re.IGNORECASE)
            fillup_df['question'].append(re.sub(reg, "___________", question_df.final_text.iloc[i]))
            fillup_df['answer'].append(question_df.answer.iloc[i])
  # fillup_df=pd.DataFrame(fillup_df)
  no=min(no,len(fillup_df['question']))
  ques2=""
  for i in range(no):
    ques2+=str(i+1)+". "+str(fillup_df['question'][i])+"\n"
    ques2+="Answer: "+ str(fillup_df['answer'][i])+"\n\n"  
  return ques2
  return fillup_df['answer'][0]

def extract(filepath,choice):
    txt=read(filepath)
    txt2=' '.join(txt.split('\n'))
    ocr=" "
    summary=" "
    sentences=" "
    gaps=" "
    if 'Generate text' in choice : ocr=txt
    if 'Summary' in choice : summary=summarize(txt2)
    if 'Important Sentences' in choice : sentences=important_sentences(txt2,5)
    if 'Fill in the Gaps' in choice : gaps=fill_in_the_gaps(txt2)
    # return (txt,"dfd",', '.join(choice),type(choice))
    # return (txt,summarize(txt2),fill_in_the_gaps(txt2))
    return (ocr,summary,sentences,gaps)
    

## Creating Gradio Interface

In [6]:
choices=['Generate text','Summary','Important Sentences','Fill in the Gaps']

title = "PicSum"
description = "Gradio demo for PicSum project. You can give an image as input on the left side and then click on the submit button. The generated text, summary, important sentences and fill in the gaps would be generated on the right side."
gr.Interface(
    extract, 
    [gr.inputs.Image(type="filepath", label="Input"),gr.inputs.CheckboxGroup(choices, type="value", default=['Generate text'], label='Options') ], 
    [gr.outputs.Textbox(label="Generated Text"),
     gr.outputs.Textbox(label="Summary"),
     gr.outputs.Textbox(label="Important Sentences"),
     gr.outputs.Textbox(label="Fill in the gaps")],
    title=title,
    description=description,
    examples=[['a.png', ['Generate text']], ['b.png', ['Generate text','Summary','Important Sentences']], ]
).launch(enable_queue=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://5e30ef76dcc4ce737e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
